In [20]:
import pandas as pd
import statistics as stat
import numpy as np
from scipy.integrate import simps
from scipy.optimize import curve_fit
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import cm
import matplotlib.patches as mpl_patches
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator, AutoLocator)
import csv
import math
import matplotlib.font_manager as font_manager
from os import listdir
from os.path import isfile, join, isdir
import to_precision as tp
from math import log10, floor
import matplotlib.ticker as ticker
import re
from matplotlib.gridspec import GridSpec
import PIL
from matplotlib.image import imread
from tempfile import NamedTemporaryFile

In [21]:
# Single exponential function to fit
def Kinetic(t, kobs, Seq, s0):
    return(s0 + ((Seq - s0)* np.exp(-kobs * t)))
# Double exponential function to fit
def Kinetic2(t, kobs, kobs2, F, s0, Seq):
    return(s0 + F*((Seq - s0)* np.exp(-kobs * t))+(1-F)*((Seq - s0)* np.exp(-kobs2 * t)))

# Get current size of overall figure
def get_size(fig, dpi=600):
    with NamedTemporaryFile(suffix='.png') as f:
        fig.savefig(f.name, bbox_inches='tight', dpi=dpi)
        height, width, _channels = imread(f.name).shape
        return width / dpi, height / dpi

# Set size of overall figure    
def set_size(fig, size, dpi=600, eps=1e-2, give_up=2, min_size_px=10):
    target_width, target_height = size
    set_width, set_height = target_width, target_height # reasonable starting point
    deltas = [] # how far we have
    while True:
        fig.set_size_inches([set_width, set_height])
        actual_width, actual_height = get_size(fig, dpi=dpi)
        set_width *= target_width / actual_width
        set_height *= target_height / actual_height
        deltas.append(abs(actual_width - target_width) + abs(actual_height - target_height))
        if deltas[-1] < eps:
            return True
        if len(deltas) > give_up and sorted(deltas[-give_up:]) == deltas[-give_up:]:
            return False
        if set_width * dpi < min_size_px or set_height * dpi < min_size_px:
            return False
        
        
# Round up x value to next nice*10^c and get interval for associated value
def roundupNice(x,nice = [1,1.25,1.5,2,2.5,4,5,6,7.5,8,10]):
    interval = 0
    for a in nice:
        if x >0:
            if abs(x) < 10**floor(log10(abs(x)))*a:
                rounded = 10**floor(log10(abs(x)))*a
                if a in [1,1.25,1.5,2.5,5,7.5,10]:
                    interval = 5
                elif a in [2,4,8]:
                    interval = 4
                elif a == 6:
                    interval = 6
                return rounded, interval
        elif x < 0 :
            if abs(x) > 10**floor(log10(abs(x)))*a:
                rounded = 10**floor(log10(abs(x)))*a*-1
                if a in [1,1.25,1.5,2.5,5,7.5,10]:
                    interval = 5
                elif a in [2,4,8]:
                    interval = 4
                elif a == 6:
                    interval = 6
                return rounded, interval

# Round down x value to next nice*10^c            
def roundDownNice(x,nice = [1,1.25,1.5,2,2.5,4,5,6,7.5,8,10]):
    if x >0:
        rounded =[10**floor(log10(abs(x)))*a for a in sorted(nice,reverse = True) if abs(x)>10**floor(log10(abs(x)))*a][0]
    elif x < 0:
        rounded =[10**floor(log10(abs(x)))*a*-1 for a in sorted(nice) if abs(x)<10**floor(log10(abs(x)))*a][0]
    return rounded

# Integral transformation of double exponential equation
def Fitting(Data):
    s = [0]
    for a in range(1,len(Data)):
        s.append(s[a-1]+0.5*(Data.iloc[a]+Data.iloc[a-1])*(Data.index[a]-Data.index[a-1]))
    ss = [0]
    for a in range(1,len(Data)):
        ss.append(ss[a-1]+0.5*(s[a]+s[a-1])*(Data.index[a]-Data.index[a-1]))
    S = pd.Series(s)
    SS = pd.Series(ss)

    SS2 = SS**2
    SSS = SS*S
    X = pd.Series(Data.index.values)
    X2 = X**2
    SSX2 = SS*X2
    SSX = SS*X
    S2 = S**2
    X4 = X2**2
    SX2 = S*X2
    SX = S*X
    X3 = X**3

    m11 = SS2.sum()
    m22 = S2.sum()
    m33 = X4.sum()
    m44 = X2.sum()
    m55 = len(Data)
    m12=m21 = SSS.sum()
    m13=m31 = SSX2.sum()
    m14=m41 = SSX.sum()
    m15=m51 = SS.sum()
    m23 = m32 = SX2.sum()
    m24 = m42 = SX.sum()
    m25 = m52 = S.sum()
    m34 = m43 = X3.sum()
    m35 = m53 = X2.sum()
    m45 = m54 = X.sum()

    M = np.array([[m11,m12,m13,m14,m15]\
                ,[m21,m22,m23,m24,m25]\
                ,[m31,m32,m33,m34,m35]\
                ,[m41,m42,m43,m44,m45]\
                ,[m51,m52,m53,m54,m55]])

    M_1 = np.linalg.inv(M)

    SSY = SS*Data.reset_index(drop = True)
    SY = S*Data.reset_index(drop = True)
    X2Y = X2*Data.reset_index(drop = True)
    XY = X*Data.reset_index(drop = True)
    Y = Data.reset_index(drop = True)

    N = np.array([[SSY.sum()]\
                  ,[SY.sum()]\
                  ,[X2Y.sum()]\
                  ,[XY.sum()]\
                  ,[Y.sum()]])

    M_1N = M_1.dot(N)

    A = M_1N[0][0]
    B = M_1N[1][0]
    c = B**2
    c = c+(4*A)
    p = (B+np.sqrt(c))/2
    q = (B-np.sqrt(c))/2

    A = np.exp(p*X)
    B = np.exp(q*X)
    A2 = A**2
    AB = A*B
    B2 = B**2
    P11 = m55
    P12 = P21 = A.sum()
    P13 = P31 = B.sum()
    P22 = A2.sum()
    P23 = P32 = AB.sum()
    P33 = B2.sum()
    P = np.array([[P11,P12,P13]\
                 ,[P21,P22,P23]\
                 ,[P31,P32,P33]])

    P_1 = np.linalg.inv(P)
    Q1 = Y.sum()
    AY = A*Y
    BY = B*Y

    Q = np.array([[Q1]\
                 ,[AY.sum()]\
                 ,[BY.sum()]])

    P_1Q = P_1.dot(Q)
    P_1Q
    
    a = P_1Q[0][0]
    b = P_1Q[1][0]
    c = P_1Q[2][0]
    S0 = a
    Seq = c+ a + b
    F = b/(Seq-S0)
    if -q>-p:
        k1 = -q
        k2 = -p
        F = 1-F
    elif -p>-q:
        k1 = -p
        k2 = -q
            
    parameters = [k1,k2,F,S0,Seq]
    return parameters

# Read in csv file, normalizing data from 1 to 0, and finding table based on "Time Repeat"
def ReadinCSV(directory, S0 = "Auto",Units = "Auto", Normalize = True):
    # Create dictionary for averaged data with error
    daverage = {}
    
    # Assign units to None or value    
    if Units == "Auto":
        units = None
    elif Units != "Auto":
        units = Units

    # Create list of csv files in folder        
    onlyfiles = [f for f in listdir(directory) if isfile(join(directory, f)) and f.endswith(".csv")]

    # Find biological replicate number    
    ReplicateNumber = re.match(r"([a-z]+)([0-9]+)", directory.split()[-1], re.I)
    if ReplicateNumber:
        ReplicateNumber = ReplicateNumber.groups()
        ReplicateNumber = int(ReplicateNumber[-1])
    else:
        lastterm = directory.split()[-1]
        ReplicateNumber = re.split(r"[-.]", lastterm)
        if isinstance(ReplicateNumber, list):
            ReplicateNumber= int(ReplicateNumber[0])
            
    # Iterate over each csv file            
    for k in range(len(onlyfiles)):
        # Take first number as concentration                
        outname1 = os.path.splitext(onlyfiles[k])
        outname = outname1[0].split()
        
        # Look for units in filename       
        for a in range(len(outname)):
            if Units == "Auto" and units==None:
                if outname[a] == "mM":
                    units = "mM"
                elif outname[a]== "µM" or outname[a] =="uM":
                    units = "µM"

        # Check for and remove default filenames
        namecheck = [item for item in ["Run"] if item in outname[0]]
        if len(namecheck)!=0:
            print("Found default filename ", os.path.join(directory,onlyfiles[k]))
            continue

        # Drop 0 concentration                
        if outname[0] == '0' :
            continue

        filename = os.path.join(directory, onlyfiles[k])

        # reading csv file 
        with open(filename, 'r') as csvfile: 
            # creating a csv reader object 
            csvreader = csv.reader(csvfile) 

            # extracting each data row one by one 
            rows = [row for row in csvreader]
        ends = [i for i,j in enumerate(rows) if j == "Time Repeat".split()]

        # Read in data based on position
        columns = rows[ends[0]+1]
        file = pd.read_csv(filename , index_col = 0,  skiprows = ends[0] + 1, nrows = ends[1]-ends[0]-5, keep_default_na = False)

        # Delete blank column if it exists
        if columns[len(columns)-1] == "":
            file.drop(file.columns[len(file.columns)-1], axis=1, inplace=True)

        # Drop Dead time
        truncated = file.truncate(before = 0.01)
        truncated.replace(0, np.NaN, inplace = True)
        
        # Fit each column to determine endpoints, then normalize each column
        if S0 == "Auto" and Normalize == True:
            for column in truncated:
                dropped = pd.DataFrame()
                dropped = truncated[column].dropna()
                params1, pcov = curve_fit(Kinetic, dropped.index, dropped)
                truncated[column] = (truncated[column]-params1[2])/(params1[1]-params1[2])

        # Take average and standard deviation between columns, assign number of columns and biological replicate number                    
        average = pd.DataFrame()
        average["Avg"] = truncated.mean(axis = 1)
        average["Stdev"] = truncated.std(axis =1)
        average["Count"] = truncated.count(axis=1)
        average["Replicate"] = ReplicateNumber

        # Write to dictionary containing all within a folder                
        daverage[outname[0]] = average

    return daverage, units

In [22]:
# Find all csv files based on file organization
def FolderFinder(Folder, comparison):
    if comparison == "Ion":
        # Open folders with pH values        
        IonFolders = tuple(os.path.join(Folder,x) for x in os.listdir(Folder) if os.path.isdir(os.path.join(Folder, x)) if x!= "Plots" if x != "Plots Double" if x!="Gluconate")
        
        # Open same ion with different replicates
        IonReplicates = {}
        for a in range(len(IonFolders)):
            subfolder = IonFolders[a]
            IonReplicates[os.path.basename(subfolder)] = [os.path.join(subfolder,x) for x in os.listdir(subfolder) if os.path.isdir(os.path.join(subfolder, x)) if x != "Plots Double" if x!="Plots"]

        # Assign order to plot ions           
        key_order1 = ["Chloride","Bromide","Iodide","Nitrate"]
        key_order = [x for x in key_order1 if x in IonReplicates.keys()]        
        IonReplicatesOrdered = {k : IonReplicates[k] for k in key_order}

        return IonReplicatesOrdered
    
    elif comparison == "pH":
        # Open each pH folder        
        pHFolders = tuple(os.path.join(Folder, x) for x in os.listdir(Folder) if os.path.isdir(os.path.join(Folder, x)) if x!= "Gluconate" if x!= "Plots Double" if x!= "Plots")

        # Find subfolders/Replicates within each of the Ion folders, append to Ions if not already there
        Ions=[]
        for a in range(len(pHFolders)):
            foldername = os.path.basename(pHFolders[a]).split()
            for b in foldername:
                if b == "pH":
                    subfolder = pHFolders[a]
                    for x in os.listdir(subfolder):
                        if os.path.isdir(os.path.join(subfolder,x)) and x not in Ions and x!= "Plots" and x!="Plots Double" and x!= "Gluconate":
                            Ions.append(x)

        # Group each pH based on ion                            
        IonpHvalues = {}
        for b in Ions:
            directories = []
            for a in range(len(pHFolders)):
                if os.path.isdir(os.path.join(pHFolders[a],b)):
                    directories.append(os.path.join(pHFolders[a],b))
            IonpHvalues[b] = directories

        AllIonReplicates = {}
        AllIonReplicatesOrdered = {}
        for Ion in IonpHvalues.keys():
            IonReplicates = {}
            
            # Group all ions together in another dictionary            
            for pH in IonpHvalues[Ion]:
                IonReplicates[float(os.path.dirname(pH).split()[-1])]=[os.path.join(pH,x) for x in os.listdir(pH) if os.path.isdir(os.path.join(pH,x)) if x!="Plots"]
            AllIonReplicates[Ion] = IonReplicates
            
            # Reorganize on increasing pH values            
            key_order = list(AllIonReplicates[Ion].keys())
            key_order.sort(key = float)
            AllIonReplicatesOrdered[Ion] = {k : AllIonReplicates[Ion][k] for k in key_order}
        
        return AllIonReplicatesOrdered
        
# Fit data to functions, average between technical replicates        
def DataProcessing(IonReplicates,S0 = "Auto",Units = "Auto", Normalize = True):
    # Import all data, averaging between concentrations after averaging between 3 scans
    IonsData = {}
    IonsData2 = {}
    
    for Ion in IonReplicates.keys():
        IonDataDict = {}
        IonDataDict2 = {}
        
        # Read in all files and separate into two dictionaries based on biological replicate (in filename)
        for Replicate in IonReplicates[Ion]:
            # Read in each csv file
            Data, Units = ReadinCSV(Replicate, S0,Units, Normalize)

            ReplicateNumber = {}
            ReplicateNumber = {conc:Data[conc]["Replicate"].max() for conc in Data.keys()}
            RepNumber = max(ReplicateNumber.values())
            
            # Write first biological replicate to first dictionary
            if RepNumber ==1:
                # If first replicate to be added, create dataframes for each concentration                
                if len(IonDataDict) ==0:
                    IonDataDict = {conc:pd.DataFrame(Data[conc]) for conc in Data.keys() if Data[conc]["Replicate"].max()==1}

                # Concatenate technical replicates together if first has been added                
                elif len(IonDataDict) !=0:
                    IonDataDict = {conc:pd.concat((IonDataDict[conc],Data[conc]), axis = 1) for conc in Data.keys() if Data[conc]["Replicate"].max()==1}

            # Write second biological replicate to second dictionary
            if RepNumber ==2:
                if len(IonDataDict2) ==0:
                    IonDataDict2 = {conc:pd.DataFrame(Data[conc]) for conc in Data.keys() if Data[conc]["Replicate"].max()==2}

                # Concatenate technical replicates together                
                elif len(IonDataDict2) !=0:
                    IonDataDict2 = {conc:pd.concat((IonDataDict2[conc],Data[conc]), axis = 1) for conc in Data.keys() if Data[conc]["Replicate"].max()==2}

        # Average technical replicates for first biological replicate            
        for conc in IonDataDict:
            tempavg = pd.DataFrame(IonDataDict[conc]["Avg"])
            tempstdev = pd.DataFrame(IonDataDict[conc]["Stdev"])
            tempcount = pd.DataFrame(IonDataDict[conc]["Count"])            
            tempavg2 = tempavg.dropna()
            
            # Check if time points have been dropped from inconsistent time measurements            
            if len(tempavg2)<10000:
                # Reindex to nearest time point (if within tolerance), then drop others
                for b in range(len(tempavg.columns)):
                    column_numbers = [x for x in range(tempavg.shape[1])]
                    column_numbers.remove(b)
                    df = tempavg.iloc[:, column_numbers]
                    if len(df.dropna())>12000:
                        a = column_numbers[0]
                        c = b
                        continue
                newdf = tempavg.iloc[:,a].dropna()
                tempavg1 = pd.DataFrame(index = newdf.index)
                tempstdev1 = pd.DataFrame(index = tempavg1.index)
                tempcount1 = pd.DataFrame(index = tempavg1.index)
                tempavg = tempavg.truncate(after = tempavg.iloc[:,c].index[-1])
                tempstdev = tempstdev.truncate(after = tempavg1.index[-1])
                tempcount = tempcount.truncate(after = tempavg1.index[-1])
                # Reindex with set tolerance
                for b in range(len(tempavg.columns)):
                    tempavg1[b] = tempavg.iloc[:,b].dropna().reindex(tempavg1.index, method = 'nearest', tolerance = 0.05)
                    tempstdev1[b] = tempstdev.iloc[:,b].dropna().reindex(tempstdev1.index, method = 'nearest', tolerance = 0.05)
                    tempcount1[b] = tempcount.iloc[:,b].dropna().reindex(tempcount1.index, method = 'nearest', tolerance = 0.05)

                tempavg = tempavg1
                tempstdev = tempstdev1
                tempcount = tempcount1

            tempvar = tempstdev**2
            tempavg2 = tempavg**2
            tempcount1 = tempcount-1

            # Take average and variation between technical replicates, propagating variation between with variation within being summed            
            if isinstance(tempavg,pd.DataFrame):
                if len(tempavg.columns) != 1:
                    tempvar1 = pd.Series(data = 0,index = tempavg.index)
                    for b in range(len(tempstdev.columns)):
                        tempvar1 = tempvar1+ tempcount1.iloc[:,b]*tempvar.iloc[:,b]+tempcount.iloc[:,b]*((tempavg.iloc[:,b]-tempavg.mean(axis =1))**2)
                    tempvar1 = tempvar1/(tempcount.sum(axis=1)-1)
                    tempavg = tempavg.mean(axis = 1)

                    daverage = pd.DataFrame(index = IonDataDict[conc].index)
                    daverage["Avg"] = tempavg
                    daverage.astype({'Avg': np.float128}).dtypes
                    
                    daverage["Stdev"] = tempvar1**0.5
                    daverage.astype({'Stdev': np.float128}).dtypes
                    # Keep track of number of injections total                    
                    daverage["Count"] = tempcount.sum(axis=1)

                    IonDataDict[conc] = daverage
                    
        # Average technical replicates for second biological replicate                    
        if len(IonDataDict2) !=0:
            for conc in IonDataDict2:
                tempavg = pd.DataFrame(IonDataDict2[conc]["Avg"])
                tempstdev = pd.DataFrame(IonDataDict2[conc]["Stdev"])
                tempcount = pd.DataFrame(IonDataDict2[conc]["Count"])

                tempvar = tempstdev**2
                tempavg2 = tempavg**2
                tempcount1 = tempcount-1

                if isinstance(tempavg,pd.DataFrame):
                    if len(tempavg.columns) != 1:
                        tempvar1 = pd.Series(data = 0,index = tempavg.index)
                        for b in range(len(tempstdev.columns)):
                            tempvar1 = tempvar1+ tempcount1.iloc[:,b]*tempvar.iloc[:,b]+tempcount.iloc[:,b]*((tempavg.iloc[:,b]-tempavg.mean(axis =1))**2)
                        tempvar1 = tempvar1/(tempcount.sum(axis=1)-1)
                        tempavg = tempavg.mean(axis = 1)

                        daverage = pd.DataFrame(index = IonDataDict2[conc].index)
                        daverage["Avg"] = tempavg
                        daverage.astype({'Avg': np.float128}).dtypes

                        daverage["Stdev"] = tempvar1**0.5
                        daverage.astype({'Stdev': np.float128}).dtypes
                        
                        daverage["Count"] = tempcount.sum(axis=1)
                        daverage.astype({'Count': np.float128}).dtypes

                        IonDataDict2[conc] = daverage
                        
        if isinstance(Ion, str):            
            IonsData[os.path.basename(Ion)] = IonDataDict
            IonsData2[os.path.basename(Ion)] = IonDataDict2           
        elif isinstance(Ion, float):
            IonsData[Ion] = IonDataDict
            IonsData2[Ion] = IonDataDict2

    # Fit linear kinetic function vs concentration to check for outliers in calculated parameters
    ddfavg = {}
    Slope = pd.DataFrame()

    # First biological replicate 
    for Ion in IonsData.keys():
        dfavg = pd.DataFrame()
        calculated = pd.DataFrame()
        Lowerbound = ()
        Upperbound = ()
        # Calculate initial parameters for each concentration based on Fitting()        
        for conc in IonsData[Ion].keys():
            IonsData[Ion][conc].dropna(axis=0, inplace = True)
            
            # If S0 == Auto fit by allowing both S0 and Seq to vary 
            if S0 == "Auto":
                Data= pd.Series(IonsData[Ion][conc]["Avg"], index = IonsData[Ion][conc].index)
                out = Fitting(Data)
                calculated[int(conc)] = out

        # Determine correlation of concentrations                
        correlation_matrix = np.corrcoef(calculated.columns.values, calculated.iloc[0,:])
        correlation_xy = correlation_matrix[0,1]
        r_squared = correlation_xy**2
        conc1 = None
        df = pd.DataFrame()
        df1 = pd.DataFrame()
        
        # Fit data to find rates        
        def linearfit(x,m,b):
            return m*x+b
        param,pcov = curve_fit(linearfit, calculated.columns.values, calculated.iloc[0])
        slope, intercept = param[0], param[1]        
        
        # Check for outliers        
        for col in calculated.columns:
            df[col] = calculated[col]                    
        if r_squared<0.95:
            r_squared1 = [0.99, 0.95, 0.9]
            for r in r_squared1:
                # Find if there is one outlier by dropping each concentration               
                for conc in df.columns:
                    df1 = df.drop(columns = conc)
                    correlation_matrix = np.corrcoef(df1.columns.values, df1.iloc[0])
                    correlation_xy = correlation_matrix[0,1]
                    r_squared = correlation_xy**2
                    param, pcov = curve_fit(linearfit, df1.columns.values, df1.iloc[0])
                    if r_squared >r and param[0]>0:
                        conc1 = conc
                        break
                        
                # Replace outlier                        
                if conc1 is not None:
                    param,pcov = curve_fit(linearfit, df1.columns.values, df1.iloc[0])
                    df[conc1].iloc[0] = linearfit(conc1,*param)
                    slope, intercept = param[0], param[1]
                    break
                    
                # Find if there are two outliers by dropping pairs of concentrations                    
                elif conc1 is None:
                    concs = [[conc1, conc2] for conc1 in df.columns for conc2 in df.columns]
                    for conc in concs:
                        df1 = df.drop(columns = conc)
                        correlation_matrix = np.corrcoef(df1.columns.values, df1.iloc[0])
                        correlation_xy = correlation_matrix[0,1]
                        r_squared = correlation_xy**2
                        param, pcov = curve_fit(linearfit, df1.columns.values, df1.iloc[0])                        
                        if r_squared >r and param[0]>0:
                            conc1 = conc
                            break
                            
                # Replace both outliers                            
                if conc1 is not None:
                    param,pcov = curve_fit(linearfit, df1.columns.values, df1.iloc[0])
                    param1, pcov1 = curve_fit(linearfit, df1.columns.values, df1.iloc[1])
                    for conc in conc1:
                        df[conc].iloc[0] = linearfit(conc,*param)
                        df[conc].iloc[1] = linearfit(conc, *param1)
                    slope, intercept = param[0], param[1]
                    break                            
        calculated = df        
        Slope[Ion] = [slope, intercept]
        
        for conc in IonsData[Ion].keys():
            
            # Set bounds of fitting to allow 15% variation from predicted for kobs1, and physical bounds on other parameters           
            Lowerbound = [x-(0.15*abs(x)) for x in calculated[int(conc)]]
            Upperbound = [x+(0.15*abs(x)) for x in calculated[int(conc)]]
            Lowerbound[2] = 0
            Lowerbound[3] = -1
            Lowerbound[4] = -1
            Upperbound[2] = 1
            Upperbound[3] = 2
            Upperbound[4] = 2
            tuple(Lowerbound)
            tuple(Upperbound)

            # Fit data with weighted regression, using output of Fitting() as initial guesses          
            kobs, pcov = curve_fit(Kinetic2, IonsData[Ion][conc].index, IonsData[Ion][conc]["Avg"], sigma = IonsData[Ion][conc]["Stdev"], absolute_sigma=True, maxfev = 10000000,p0 = calculated[int(conc)], bounds = (Lowerbound,Upperbound))
#                 kobs, pcov = curve_fit(Kinetic2, IonsData[Ion][conc].index, IonsData[Ion][conc]["Avg"], maxfev = 10000000,p0 = out)
            true_value_of_y = IonsData[Ion][conc]["Avg"]
            predicted_value_of_y = Kinetic2(IonsData[Ion][conc].index,*kobs)
            MSE = np.square(np.subtract(true_value_of_y,predicted_value_of_y)).mean()
            err = [np.sqrt(x) for x in np.diag(pcov)]
            kobs1 = [x for x in kobs]
            kobs1.extend(err)
            kobs1.append(MSE)
            dfavg[int(conc)]= kobs1
            
        # Determine final on rate            
        param, pcov = curve_fit(linearfit, dfavg.columns.values, dfavg.iloc[0])
        Slope[Ion].iloc[0] = param[0]

        # Write fits to dictionary                    
        dfavg.rename(index = {0:"kobs", 1:"kobs2", 2:"Frac",3:"S0",4:"Seq",5:"kobserr",6:"kobs2err",7:"Fracerr",8:"S0err",9:"Seqerr",10:"MSE"}, inplace=True)
        dfavg.sort_index(axis=1, inplace = True)
        ddfavg[Ion] = dfavg
        
    # Check second biological replicate
    ddfavg2 = {}
    for Ion in IonsData2.keys():
        dfavg = pd.DataFrame()
        calculated = pd.DataFrame()
        Lowerbound = ()
        Upperbound = ()
        # Calculate initial parameters for each concentration based on Fitting()
        for conc in IonsData2[Ion].keys():
            IonsData2[Ion][conc].dropna(axis=0, inplace = True)
            
            # If S0 == Auto fit by allowing both S0 and Seq to vary 
            if S0 == "Auto":
                Data= pd.Series(IonsData2[Ion][conc]["Avg"], index = IonsData2[Ion][conc].index)
                out = Fitting(Data)
                calculated[int(conc)] = out

        # Check correlation of biological replicate
        correlation_matrix = np.corrcoef(calculated.columns.values, calculated.iloc[0,:])
        correlation_xy = correlation_matrix[0,1]
        r_squared = correlation_xy**2
        conc1 = None
        df = pd.DataFrame()
        df1 = pd.DataFrame()
        
        # Determine kinetic rate of replicate        
        param,pcov = curve_fit(linearfit, calculated.columns.values, calculated.iloc[0])
        slope, intercept = param[0], param[1]        
        
        # Determine if there is an outlier in replicate        
        for col in calculated.columns:
            df[col] = calculated[col]                    
        if r_squared<0.95:
            r_squared1 = [0.99, 0.95, 0.9]
            for r in r_squared1:
                # Determine if there is one outlier                
                for conc in df.columns:
                    df1 = df.drop(columns = conc)
                    correlation_matrix = np.corrcoef(df1.columns.values, df1.iloc[0])
                    correlation_xy = correlation_matrix[0,1]
                    r_squared = correlation_xy**2
                    param, pcov = curve_fit(linearfit, df1.columns.values, df1.iloc[0])
                    if r_squared >r and param[0]>0:
                        conc1 = conc
                        break
                # Replace outlier if it exists                        
                if conc1 is not None:
                    param,pcov = curve_fit(linearfit, df1.columns.values, df1.iloc[0])
                    df[conc1].iloc[0] = linearfit(conc1,*param)
                    slope, intercept = param[0], param[1]
                    break
                    
                # Determine if there are two outliers                    
                elif conc1 is None:
                    concs = [[conc1, conc2] for conc1 in df.columns for conc2 in df.columns]
                    for conc in concs:
                        df1 = df.drop(columns = conc)
                        correlation_matrix = np.corrcoef(df1.columns.values, df1.iloc[0])
                        correlation_xy = correlation_matrix[0,1]
                        r_squared = correlation_xy**2
                        param, pcov = curve_fit(linearfit, df1.columns.values, df1.iloc[0])
                        if r_squared >r and param[0]>0:
                            conc1 = conc
                            break
                            
                # Replace both outliers                            
                if conc1 is not None:
                    param,pcov = curve_fit(linearfit, df1.columns.values, df1.iloc[0])
                    param1, pcov1 = curve_fit(linearfit, df1.columns.values, df1.iloc[1])
                    for conc in conc1:
                        df[conc].iloc[0] = linearfit(conc,*param)
                        df[conc].iloc[1] = linearfit(conc, *param1)
                    slope, intercept = param[0], param[1]
                    break
        conc1 = None

        # Check correlation between replicates        
        if abs(slope-Slope[Ion][0])/Slope[Ion][0] > 0.4:
            def intercept(x,b):
                return Slope[Ion][0]*x+b
            
            # Determine if one concentration can be dropped to have correlation
            for conc in df.columns:
                df1 = df.drop(columns = conc)
                yint, pcov = curve_fit(intercept, df1.columns.values, df1.iloc[0])
                residuals = df1.iloc[0]- intercept(df1.columns.values, yint)
                ss_res = np.sum(residuals**2)
                ss_tot = np.sum((df1.iloc[0]-np.mean(df1.iloc[0]))**2)
                r_squared = 1 - (ss_res / ss_tot)
                if r_squared>0.9:
                    conc1 = conc
                    break
            if conc1 is not None:
                df[conc1].iloc[0] = intercept(conc1,yint)
                
            # Determine if two can be dropped to have correlation                
            elif conc1 is None:
                concs = [[conc1, conc2] for conc1 in df.columns for conc2 in df.columns]
                for conc in concs:
                    df1 = df.drop(columns = conc)
                    yint, pcov = curve_fit(intercept, df1.columns.values, df1.iloc[0])
                    residuals = df1.iloc[0]- intercept(df1.columns.values, yint)
                    ss_res = np.sum(residuals**2)
                    ss_tot = np.sum((df1.iloc[0]-np.mean(df1.iloc[0]))**2)
                    r_squared = 1 - (ss_res / ss_tot)
                    if r_squared>0.9:
                        conc1 = conc
                        break
                if conc1 is not None:
                    param1, pcov1 = curve_fit(linearfit, df1.columns.values, df1.iloc[1])
                    for conc in df.columns:
                        df[conc].iloc[0] = intercept(conc,yint)
                        df[conc].iloc[1] = linearfit(conc, *param1)
                        
            # Determine if three can be dropped for correlation                        
            if conc1 is None:
                concs = [[conc1,conc2,conc3] for conc1 in df.columns for conc2 in df.columns for conc3 in df.columns]
                for conc in concs:
                    df1 = df.drop(columns = conc)
                    yint, pcov = curve_fit(intercept, df1.columns.values, df1.iloc[0])
                    residuals = df1.iloc[0]- intercept(df1.columns.values, yint)
                    ss_res = np.sum(residuals**2)
                    ss_tot = np.sum((df1.iloc[0]-np.mean(df1.iloc[0]))**2)
                    r_squared = 1 - (ss_res / ss_tot)
                    if r_squared>0.9:
                        conc1 = conc
                        break
                if conc1 is not None:
                    param1, pcov1 = curve_fit(linearfit, df1.columns.values, df1.iloc[1])
                    for conc in conc1:
                        df[conc].iloc[0] = intercept(conc,yint)
                        df[conc].iloc[1] = linearfit(conc, *param1)
            param,pcov = curve_fit(linearfit, df.columns.values,df.iloc[0])
            
        # Change initial guesses based on above        
        calculated = df
        
        for conc in IonsData2[Ion].keys():
            # Set bounds for second biological replicate, limiting kobs1 to 15%, kobs2 to 40%, and others to physical bounds.           
            Lowerbound = [x-(0.15*abs(x)) for x in calculated[int(conc)]]
            Upperbound = [x+(0.15*abs(x)) for x in calculated[int(conc)]]
            Lowerbound[1] = calculated[int(conc)].iloc[1]-(.4*abs(calculated[int(conc)].iloc[1]))
            Upperbound[1] = calculated[int(conc)].iloc[1]+(.4*abs(calculated[int(conc)].iloc[1]))
            Lowerbound[2] = 0
            Lowerbound[3] = -1
            Lowerbound[4] = -1
            Upperbound[2] = 1
            Upperbound[3] = 2
            Upperbound[4] = 2
            tuple(Lowerbound)
            tuple(Upperbound)
            
            # Fit second biological replicate            
            kobs, pcov = curve_fit(Kinetic2, IonsData2[Ion][conc].index, IonsData2[Ion][conc]["Avg"], sigma = IonsData2[Ion][conc]["Stdev"], absolute_sigma=True, maxfev = 10000000,p0 = calculated[int(conc)], bounds = (Lowerbound,Upperbound))
            true_value_of_y = IonsData2[Ion][conc]["Avg"]
            predicted_value_of_y = Kinetic2(IonsData2[Ion][conc].index,*kobs)
            MSE = np.square(np.subtract(true_value_of_y,predicted_value_of_y)).mean()
            err = [np.sqrt(x) for x in np.diag(pcov)]
            kobs1 = [x for x in kobs]
            kobs1.extend(err)
            kobs1.append(MSE)

            dfavg[int(conc)]= kobs1

        # Write fits to dictionary                    
        dfavg.rename(index = {0:"kobs", 1:"kobs2", 2:"Frac",3:"S0",4:"Seq",5:"kobserr",6:"kobs2err",7:"Fracerr",8:"S0err",9:"Seqerr", 10: "MSE"}, inplace=True)
        dfavg.sort_index(axis=1, inplace = True)
        ddfavg2[Ion] = dfavg

    IonsDataReplicates = {1: IonsData, 2: IonsData2}
    ddfavgReplicates = {1: ddfavg, 2: ddfavg2}

    return IonsDataReplicates, ddfavgReplicates, Units

In [23]:
def PlotKinetic(Data, DataType,output,key,Marker, MarkerStyle, LineStyle, Units,Data2 = None, Errorbar = True, ion = False, Ylim = "Uniform", S0 = "Auto", size = (3.4,2), DPI = 600):
    # Plot pseudo first order kinetics    
    if DataType == "Kinetic" or DataType =="kinetic":
        # Adjust default font and style
        mpl.rcParams['font.family'] = 'Arial'
        plt.rcParams['font.size'] = 10
        plt.rcParams['axes.linewidth'] = 1
        plt.rcParams['mathtext.default'] = "regular"

        # Define functions to fit 
        def fit(c, k2, k3):
            return(k2*c+k3)
        def fit2(C, K1, K_1, K2, K_2):
            return((K2*C)/(K_1/K1+C)+K_2)

        # Iterate biological replicates separately        
        for Replicate in Data.keys():
            dkineticparam={}
            dkineticparam2 = {}
            dkineticparams={}
            PlotOrder = []
            # Check to make sure enough colors are in key   
            if len(Data[Replicate].keys())>len(key):
                print("Why are you plotting more than 6 lines? Code more colors.")
                return
            
            # Initialize figure, figsize = size of plot (not including axis labels)         
            fig = plt.figure(figsize = (2.45,1))
            ax = fig.add_axes([0, 0, 1, 1])

            # Set axes major and minor tick marks
            ax.xaxis.set_tick_params(which='minor', size=4, width=1, direction='inout')
            ax.xaxis.set_tick_params(which='major', size=8, width=1, direction='inout')
            ax.yaxis.set_tick_params(which='minor', size=4, width=1, direction='inout')
            ax.yaxis.set_tick_params(which='major', size=8, width=1, direction='inout')

            # Iterate adding linear fits to plot        
            maxYvalue = []
            for Ion in Data[Replicate].keys():
                PlotOrder.append(Ion)
                
                # Plot kobs1 vs concentration
                if Errorbar == True:
                    ax.errorbar(Data[Replicate][Ion].columns.values, Data[Replicate][Ion].loc["kobs"],yerr = Data[Replicate][Ion].loc["kobserr"], color = key[Ion], marker=Marker[Ion],ls= 'None', fillstyle=MarkerStyle[Ion], ms=6)
                elif Errorbar == False:
                    ax.plot(Data[Replicate][Ion].columns.values, Data[Replicate][Ion].loc["kobs"],  color = key[Ion], marker=Marker[Ion],ls= 'None', fillstyle=MarkerStyle[Ion], ms=6)

                # Fit linear function to get on and off rates
                concentrations = [float(value) for value in Data[Replicate][Ion].columns.values]
                kineticrate,pcov = curve_fit(fit,Data[Replicate][Ion].columns.values,Data[Replicate][Ion].loc["kobs"], absolute_sigma=True, sigma = Data[Replicate][Ion].loc["kobserr"], maxfev = 100000)

                # Plot Fit
                num_range = range(0, int(Data[Replicate][Ion].columns.max())+1)
                ax.plot(num_range, fit(num_range, *kineticrate), linestyle=LineStyle[Ion], linewidth=0.5, color=key[Ion])

                # Record max y value to set y-limit later                
                maxYvalue.append(max(fit(num_range, *kineticrate)))
                
                # Determine error in k1 and k-1                
                ek1 = np.sqrt(np.diag(pcov)[0])
                ek_1 = np.sqrt(np.diag(pcov)[1])
                
                # Calculate correlation coefficient
                residuals = Data[Replicate][Ion].loc["kobs"]- fit(Data[Replicate][Ion].columns.values, *kineticrate)
                ss_res = np.sum(residuals**2)
                ss_tot = np.sum((Data[Replicate][Ion].loc["kobs"]-np.mean(Data[Replicate][Ion].loc["kobs"]))**2)
                r_squared = 1 - (ss_res / ss_tot)

                # Convert k1 to M^-1                
                if Units == "mM":
                    kineticrate[0] = kineticrate[0]*1000
                    ek1 = ek1*1000
                    Units1 = "M"
                elif Units == "µM":
                    kineticrate[0] = kineticrate[0]*1e6
                    ek1 = ek1*1e6
                    Units1 = "M"
                else:
                    Units1 = Units

                # If there is no error, set it to one millionth                    
                if ek1 == 0:
                    ek1 = 1*10^(-9)
                if ek_1 == 0:
                    ek_1 = 1*10^(-9)

                # Round error to one sig fig for legend
                ek1 = tp.std_notation(ek1,1)
                ek_1 = tp.std_notation(ek_1, 1)
                # Determine number of digits of rounded error
                digitsk1 = -int(floor(log10(abs(float(ek1)))))+1
                digitsk_1 = -int(floor(log10(abs(float(ek_1)))))+1
                # Store values
                kineticparam = [kineticrate[0], kineticrate[1], ek1, ek_1, digitsk1, digitsk_1, r_squared]
                dkineticparam[Ion] = kineticparam

                # Create strings for legend, rounding k1 and k-1 to same decimal places as error           
                if dkineticparam[Ion][4]>0:
                    k1r = format(round(dkineticparam[Ion][0], dkineticparam[Ion][4]),'.{}f'.format(dkineticparam[Ion][4]))
                    k1s = k1r + " ± "+ dkineticparam[Ion][2] + " " + Units1
                else:
                    k1r = int(round(dkineticparam[Ion][0], dkineticparam[Ion][4]))
                    k1s = str(k1r) +" ± "+ dkineticparam[Ion][2] + " "+ Units1

                if dkineticparam[Ion][5]>0:
                    k_1r = format(round(dkineticparam[Ion][1], dkineticparam[Ion][5]),'.{}f'.format(dkineticparam[Ion][5]))
                    k_1s = k_1r + " ± "+ dkineticparam[Ion][3]+ " s"
                else:
                    k_1r = int(round(dkineticparam[Ion][1], dkineticparam[Ion][5]))
                    k_1s = str(k_1r) +" ± "+ dkineticparam[Ion][3] + " s"
                kineticparams = [k1s, k_1s, r_squared]
                dkineticparams[Ion] = kineticparams

            # Set x-axis labels based on ion
            if ion != False and ion != None:
                label = ion
                if ion == "Chloride":
                    label = "\mathdefault{Cl}\mathrm{^{-}}"
                elif ion == "Bromide":
                    label = "\mathdefault{Br}\mathrm{^{-}}"
                elif ion == "Nitrate":
                    label = "\mathdefault{NO}\mathrm{_\mathdefault{3}^{-}}"
                elif ion == "Iodide":
                    label = "\mathdefault{I}\mathrm{^{-}}"
                
                x_label = "[$"+label + "$] (" + Units + ")"
            elif ion == False or ion == None:
                x_label = "Concentration (%s)"%(Units)

            # Set axes labels           
            ax.set_xlabel(x_label, labelpad = 8)
            ax.set_ylabel('$\mathit{k}\mathrm{_{{\mathdefault{obs}}}}$ ($\mathdefault{s}\mathrm{^{{\mathdefault{-1}}}}$)', fontstyle = "normal",labelpad = 8)

            # create a list with three empty handles for legend (3 per line plotted)
            handles = [mpl_patches.Rectangle((0, 0), 1, 1, fc="white", ec="white", 
                                             lw=0, alpha=0)] * 3*len(dkineticparams)
            # create the corresponding number of labels (= the text you want to display in legend)
            labels = []
            # k1, k-1, r^2 in legend
            for Ion in dkineticparams.keys():
                labels.append("$k$$_{{1}}$ = {}$^{{-1}}$ s$^{{-1}}$".format(dkineticparams[Ion][0]))
                labels.append("$k$$_{{-1}}$ = {}$^{{-1}}$".format(dkineticparams[Ion][1]))
                labels.append(dkineticparams[Ion][2])

            # create the legend, supressing the blank space of the empty line symbol and the
            # padding between symbol and label by setting handlelenght and handletextpad
            font = font_manager.FontProperties(family = 'Arial', size = 10, style='normal')
#             ax.legend(handles, labels, bbox_to_anchor = (1.04, 0.5), loc='center left', fontsize=18, 
#                       fancybox=True, framealpha=0, 
#                       handlelength=0, handletextpad=0, prop = font)

            # Change the legend font color to same color as plots
#             leg = ax.get_legend()        
#             for b in range(len(PlotOrder)):
#                 leg.texts[3*b+0].set_color(key[PlotOrder[b]])
#                 leg.texts[3*b+1].set_color(key[PlotOrder[b]])
#                 leg.texts[3*b+2].set_color(key[PlotOrder[b]])

            # Create csv file to write k1, k-1 parameters to 
            outputdict = os.path.join(os.path.dirname(output), "Rates", ion + " Replicate "+str(Replicate)+".csv")
            a_file = open(outputdict, "w")
            # Write to csv file
            writer = csv.writer(a_file)
            for k, value in dkineticparams.items():
                writer.writerow([k, [x for x in value]])

            a_file.close()
            
            # Set limits and intervals based on rounding up values        
            rightlim, interval = roundupNice(pd.concat(Data[Replicate]).columns.max())
            uplim, yInterval = roundupNice(max(maxYvalue))
            # Set lower limits to 0 and upper limits to those determined
            plt.xlim(0,rightlim)
            plt.ylim(0,uplim)
            ax.relim()
            
            # Set tick mark frequency          
            ax.xaxis.set_major_locator(ticker.MaxNLocator(interval))        
            ax.yaxis.set_major_locator(ticker.MaxNLocator(yInterval))

            if isinstance(Ion, str):
                output1 = output + Ion +" kobs.png"
            elif isinstance(Ion, float):
                output1 = output + " Replicate " + str(Replicate)+ ".tiff"
            # Create generic filename for csv files 
            output2 = os.path.join(os.path.dirname(output1),ion)
            
            # Write parameters of kinetic fits to csv files            
            for item in Data[Replicate].keys():
                output3 = output2 + " pH " + str(item)+ " Replicate " + str(Replicate)+ ".csv"
                Data[Replicate][item].to_csv(output3)
                
            # Set size of full figure                
#             set_size(fig, size, dpi = DPI)

            # Save figure            
            plt.savefig(output1, dpi=DPI,format = "tiff", transparent=True, bbox_inches='tight')
            plt.show()
            plt.close("all")       
 

In [30]:
def PlotOverlayAll(Data,Data2,output, units, size = (3.4,2.5), DPI = 600):
    # Separate biological replicates    
    for replicate in Data.keys():
        # Separate ions        
        for item in Data[replicate].keys():
            output1 = output + str(item)+ " Replicate "+str(replicate)
            # Set default formats
            plt.rcParams['mathtext.default'] = 'rm'
            mpl.rcParams['font.family'] = 'Arial'
            plt.rcParams['font.size'] = 10
            plt.rcParams['axes.linewidth'] = 1
            font = font_manager.FontProperties(family = 'Arial', size = 10, style='normal')

            # Create figure
            fig = plt.figure(figsize = (2.49,2.32))
#             fig.suptitle(os.path.basename(output) + " "+ item, fontsize=10, y =0.9, x=0.9, ha = "right",va= "bottom", fontproperties = font)
            # Initialize subplots
            gs = GridSpec(2, 1, height_ratios=[3, 1], hspace=0.05)
            ax1 = fig.add_subplot(gs[0])
            ax2 = fig.add_subplot(gs[1])
            
            # Create list of concentrations to sort in increasing value
            conci = [int(x) for x in Data[replicate][item].keys()]
            conci.sort()
            residualsdf = pd.DataFrame()
            residualdf = pd.DataFrame()
            
            # Separate highest concentration from rest
            concentrations = [str(x) for x in conci]
            concentration = concentrations[:-1]

            itemlabel = None
            if isinstance(item, float):
                itemlabel = str(item)
            elif isinstance(item, str):
                itemlabel = item
                
            # Iterate plotting concentrations (other than highest)
            for conc in concentration:
                # Calculate residual at each time point                
                residuals = Data[replicate][item][conc]["Avg"] - Kinetic2(Data[replicate][item][conc].index.values, Data2[replicate][item][int(conc)].iloc[0],\
                                                                          Data2[replicate][item][int(conc)].iloc[1],Data2[replicate][item][int(conc)].iloc[2],\
                                                                          Data2[replicate][item][int(conc)].iloc[3],Data2[replicate][item][int(conc)].iloc[4])
                # Calculate Mean Squared Error
                MSE = np.square(residuals).mean()
                # Store residuals                
                residualdf[conc] = residuals

                # Plot raw data
                ax1.plot(Data[replicate][item][str(conc)].index.values, Data[replicate][item][str(conc)]["Avg"], color = "k", marker='o',ls= 'None', markerfacecolor='None', ms=.1)

                # Option to plot fit                
                # Plot Fit
#                 ax1.plot(Data[item][str(conc)].index.values, Kinetic2(Data[item][str(conc)].index.values, Data2[item][int(conc)].iloc[0],Data2[item][int(conc)].iloc[1],Data2[item][int(conc)].iloc[2],Data2[item][int(conc)].iloc[3],Data2[item][int(conc)].iloc[4]), \
#                          linestyle='--', linewidth=1, color="blue", label = str(conc)+" "+units+" "+item)

                # Plot residuals on subplot
                ax2.plot(residuals.index.values, residuals, color = "k", marker='o',ls= 'None', markerfacecolor='None', ms=.1, label = str(conc)+" "+units+" "+itemlabel)
                # Store mean squared error
                residualsdf[conc] = MSE

            # Take highest concentration and repeat                
            conc = concentrations[-1]
            residuals = Data[replicate][item][conc]["Avg"] - Kinetic2(Data[replicate][item][str(conc)].index.values, Data2[replicate][item][int(conc)].iloc[0],\
                                                                      Data2[replicate][item][int(conc)].iloc[1],Data2[replicate][item][int(conc)].iloc[2],\
                                                                      Data2[replicate][item][int(conc)].iloc[3],Data2[replicate][item][int(conc)].iloc[4])
            # Store values
            residualdf[conc] = residuals
            MSE = np.square(residuals).mean()


            # Plot data for highest concentration in red
            ax1.plot(Data[replicate][item][str(conc)].index.values, Data[replicate][item][str(conc)]["Avg"], color = "r", marker='o',ls= 'None', markerfacecolor='None', ms=.1)

            # Plot Fit
    #         ax1.plot(Data[item][str(conc)].index.values, Kinetic2(Data[item][str(conc)].index.values, Data2[item][int(conc)].iloc[0],Data2[item][int(conc)].iloc[1],Data2[item][int(conc)].iloc[2],Data2[item][int(conc)].iloc[3],Data2[item][int(conc)].iloc[4]), \
    #                  linestyle='--', linewidth=1, color="red", label = str(conc)+" "+units+" "+item)

            # Plot residual for highest concentration in red on subplot
            ax2.plot(residuals.index.values, residuals, color = "r", marker='o',ls= 'None', markerfacecolor='None', ms=.1, label = str(conc)+" "+units+" "+itemlabel)
            # Store mean square error
            residualsdf[conc] = MSE

            # Set axes major and minor tick marks
            ax1.xaxis.set_tick_params(which='minor', size=4, width=1, direction='inout')
            ax1.xaxis.set_tick_params(which='major', size=8, width=1, direction='inout')
            ax1.yaxis.set_tick_params(which='minor', size=4, width=1, direction='inout')
            ax1.yaxis.set_tick_params(which='major', size=8, width=1, direction='inout')
            # Remove x axis ticks from main plot
            ax1.xaxis.set_major_formatter(plt.NullFormatter())
            ax1.xaxis.set_ticks_position('none')
            
            # Set y label for main plot and x range
            ax1.set_ylabel("Rel. Emission Intensity", labelpad = 8)
            ax1.set_xlim(0, Data[replicate][item][str(min(conci))].index.values[-1])
            
            # Manually set x range if needed            
#             ax1.set_xlim(0,130)

            # Set x axis tick parameters for residual subplot
            ax2.xaxis.set_tick_params(which='minor', size=4, width=1, direction='inout')
            ax2.xaxis.set_tick_params(which='major', size=8, width=1, direction='inout')
            ax2.yaxis.set_tick_params(which='minor', size=4, width=1, direction='inout')
            ax2.yaxis.set_tick_params(which='major', size=8, width=1, direction='inout')
            # Set x axis labels
            ax2.set_xlabel("Time (s)", labelpad = 8)
            ax2.set_ylabel("Residuals", labelpad = 8)

            # Calculate y range to round to            
            MinYlim = roundDownNice(residualdf.truncate(before = 2.0).min().min())
            MaxYlim, interval = roundupNice(residualdf.truncate(before = 2).max().max())
            # Make y range symmetric
            MaxYlim = max(MaxYlim, abs(MinYlim))
            MinYlim = -MaxYlim

            # Set x axis range to be the same as plot for raw data
            ax2.set_xlim(0, Data[replicate][item][str(min(conci))].index.values[-1])
            
            # Set manually if desired            
#             ax2.set_xlim(0, 130)

            # Set y range for residual plot and set ticks on edges
            ax2.set_ylim(MinYlim,MaxYlim)
            Yticks = [MinYlim, 0, MaxYlim]
            ax2.yaxis.set_major_locator(ticker.LinearLocator(3))
            # Move ticks and labels to the right side for residual plot          
            ax2.yaxis.set_label_position("right")
            ax2.yaxis.tick_right() 

            # Fix for right aligning on the right side, based on measuring width of widest label when left aligned            
            # Draw plot to have current tick label positions
            plt.draw()
            # Read max width of tick labels
            ytickcoord = max([yticks.get_window_extent(renderer = plt.gcf().canvas.get_renderer()).width for yticks in ax2.get_yticklabels()])
            # Change ticks to right aligned
            ax2.axes.set_yticklabels(ax2.yaxis.get_majorticklabels(),ha = "right")
            # Add max width of tick labels to shift ticks to proper position
            ax2.yaxis.set_tick_params(pad=ytickcoord+1)

            # Set output filetype            
            Overlay = output1 + ".tiff"

            # Set figure based on overall size if desired            
#             set_size(fig, size, dpi = DPI)

            # Save figure
            plt.savefig(Overlay, dpi=DPI, format = "tiff",transparent=True, bbox_inches='tight')
#             plt.show()
            plt.close("all")






In [25]:
def Kinetics(Folder, individual = False, comparison = "Auto",key = False,Units = "Auto",S0 = "Auto", Normalize = True, Errorbar = True, overlay = False):

    # Determine how to compare data and set output folder    
    if comparison == "Auto":
        basename = os.path.basename(Folder)
        for a in basename.split():
            if a == "pH":
                comparison = "Ion"
                outname = os.path.basename(Folder)
                if os.path.isdir(os.path.join(os.path.dirname(Folder),"Plots Double")):
                    output1 = os.path.join(os.path.dirname(Folder),"Plots Double", os.path.basename(Folder))
                    output = os.path.join(os.path.dirname(Folder),"Plots Double",outname)
                else:
                    print("Can't find Plots folder, saving in input folder")
                    output = outname
                break
            else:
                comparison = "pH"
                if os.path.isdir(os.path.join(Folder, "Plots Double")):
                    output = os.path.join(Folder,"Plots Double")
                else:
                    print("Can't find Plots folder, saving in input folder")
                    output = Folder                    
                break
                
    # Set parameters for plotting
    if key == False:
        if comparison == "Ion":
            key = {"Chloride" : 'black', "Bromide" : 'blue', \
                   "Iodide":'green',"Nitrate": 'cyan', \
                   "Nitrite":'magenta', "Fluoride":'chocolate'}
            Marker = {"Chloride": "o", "Bromide": "o", "Nitrate": "o", "Iodide":"o"}
            MarkerStyle = {"Chloride": "none", "Bromide": "none", "Nitrate": "none", "Iodide":"none"}
            LineStyle = {"Chloride": "-", "Bromide": "-", "Nitrate": "-", "Iodide":"-"}
        elif comparison == "pH":
            key = {6: "black", 6.5:"red",7:"blue"}
            Marker = {6: "o", 6.5: "o", 7: "o"}
            MarkerStyle = {6: "none", 6.5: "none", 7: "none"}
            LineStyle = {6: "-", 6.5: "-", 7: "-"}

    # Find folders for all replicates
    IonReplicates = FolderFinder(Folder, comparison)

    if comparison =="Ion":
        ExponentialData,FitData, Units = DataProcessing(IonReplicates, S0,Units, Normalize)
        if overlay == False and individual == False:
            output = output 
            PlotKinetic(FitData,"Kinetic",output, key, Marker, MarkerStyle, LineStyle, Units, Errorbar)
            plt.close("all")
    elif comparison == "pH":
        # Iterate over each ion
        for Ion in IonReplicates.keys():
            outname = Ion + " pH Overlay"
            output1 = os.path.join(output,Ion,outname)
            ExponentialData,FitData,Units = DataProcessing(IonReplicates[Ion], S0, Units, Normalize)
            PlotKinetic(FitData, "Kinetic",output1, key,Marker, MarkerStyle, LineStyle, Units, Errorbar, ion = Ion)
            if overlay == "All":
                output1 = os.path.join(output,Ion,Ion + " Overlaid pH ")
                PlotOverlayAll(ExponentialData,FitData,output1,Units)
            plt.close("all")
    if individual == True:
        PlotIndividual(ExponentialData, output, key, Marker, MarkerStyle, LineStyle, Units, Errorbar= Errorbar, Data2 = FitData)
    if overlay == True:
        PlotOverlay(ExponentialData,FitData, output, Units)
    plt.close("all")